# download_threads

なんJのスレッドタイトルを[KAKO LOGページ](https://hawk.5ch.net/livejupiter/kako/kako0000.html)からダウンロードする。

ダウンロードしたデータは `data` に保存される。

|パラメータ名|内容|
|:--|:--|
| `start` | ダウンロードするKAKO LOGの開始ページ |
| `end` | ダウンロードするKAKO LOGの終了ページ |

## 入力ファイル

なし

## 出力ファイル

* `data/*.tsv`: ダウンロードしたスレッドタイトルとそのレス数をタブ区切りで保存したファイル

In [ ]:
start = 0
end = 4428

In [ ]:
import os
import requests
import time
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from lxml import html

In [ ]:
def download_threads(dst_dir, start=0, end=9999):
    url_fmt = "https://hawk.5ch.net/livejupiter/kako/kako{0:04d}.html"

    if not os.path.exists(dst_dir):
        os.mkdir(dst_dir)

    session = requests.Session()
    adapter = HTTPAdapter(max_retries=Retry(total=5, backoff_factor=1.0))
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    for idx in range(start, end+1):
        url = url_fmt.format(idx)
        print("open:", url)
        
        res = session.get(url, timeout=5)
        res.encoding = res.apparent_encoding
        tree = html.fromstring(res.text)

        titles_odd = tree.xpath('//div[@class="main"]/p[@class="main_odd"]/span[@class="title"]/a')
        titles_even = tree.xpath('//div[@class="main"]/p[@class="main_even"]/span[@class="title"]/a')
        lines_odd = tree.xpath('//div[@class="main"]/p[@class="main_odd"]/span[@class="lines"]')
        lines_even = tree.xpath('//div[@class="main"]/p[@class="main_even"]/span[@class="lines"]')

        dst_path = os.path.join(dst_dir, "{0:04d}.tsv".format(idx))

        print("save as:", dst_path)
        with open(dst_path, "w") as f:
            for t, l in zip(titles_odd, lines_odd):
                f.write("{0}\t{1}\n".format(t.text, l.text))
            for t, l in zip(titles_even, lines_even):
                f.write("{0}\t{1}\n".format(t.text, l.text))
                
        time.sleep(5)

In [ ]:
download_threads("data", start=start, end=end)